I classify galaxy vs quasars in the Sloan Digital Sky Survey. I use the colors (u-g), (g-r), (r-i), (i-z) and I classify the data using tensorflow and MPLClassifier.

In [5]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from astroML.datasets import fetch_sdss_specgals
from astroML.utils.decorators import pickle_results

from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=False)

# Preparation of the data

In [6]:
# I read the data
import csv
import pandas as pd
data = pd.read_csv("galaxyquasar.csv")

#create arrays with colors
ug = data['u']-data['g']
gr = data['g']-data['r']
ri = data['r']-data['i']
iz = data['i']-data['z']

# Assign unique integers from 0 to 1 to galaxy/quasar
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
class_gq = le.fit_transform(data['class'])  #galaxy=0, quasar=1

ug = ug.to_numpy()
gr = gr.to_numpy()
ri = ri.to_numpy()
iz = iz.to_numpy()

X_data = np.array([ug, gr, ri, iz]).T
print(np.shape(X_data), np.shape(class_gq))

(50000, 4) (50000,)


In [7]:
# Scaling
data_normed = np.zeros((len(X_data), 5), dtype=np.float32)

for i in range(4):
    data_normed[:, i] = ((X_data[:,i] - X_data[:,i].mean()) /  X_data[:,i].std())

# I add the class galaxy=0, quasar=1
data_normed[:,4] = class_gq

# I use some data for the final test
data_test = data_normed[:1000]
data_normed = data_normed[1000:]

In [8]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# train-validation splitting
train, valid = train_test_split(data_normed, test_size=0.2)

# the columns 0, 1, 2, 3 are the colors; the column 4 is the class
X_train = train[:,:4]
Y_train = train[:,4]
X_valid = valid[:,:4]
Y_valid = valid[:,4]
X_test = data_test[:,:4]
Y_test = data_test[:,4]

# Neural Network

I define the Neural Network

In [9]:
# Make sure that we are starting a new model and not adding to an earlier one
keras.backend.clear_session()

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


(39200, 4)
(9800, 4)
(1000, 4)


In [10]:
# When I define the model, I need to use dense. There's no point in using convolution and pooling,
# because I have a 1D vector (with 4 features), I don't have 2D objects

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(4,)), 
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
clf = model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid))

c:\Users\marti\Anaconda\envs\astro\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1225/1225 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9759 - loss: 0.1477 - val_accuracy: 0.9834 - val_loss: 0.0719


I obtain predictions

In [11]:
y_prob = model.predict(X_test)   # Probability to be a quasar

y_class = np.zeros(len(y_prob))
for i in range(len(y_prob)):
   if y_prob[i] > 0.5:
      y_class[i] = 1   # I classify as quasars all the objects with prob>0.5

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [12]:
# confusion matrix
from sklearn.metrics import confusion_matrix
c = confusion_matrix(Y_test, y_class)

print(c)

[[848   6]
 [ 13 133]]


I can do better. What happens if I increase the number of hidden layers?

In [13]:
keras.backend.clear_session()

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(4,)), 
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
clf = model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid))

c:\Users\marti\Anaconda\envs\astro\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1225/1225 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9776 - loss: 0.0961 - val_accuracy: 0.9842 - val_loss: 0.0662


In [14]:
y_prob = model.predict(X_test)   # Probability to be a quasar

y_class = np.zeros(len(y_prob))
for i in range(len(y_prob)):
   if y_prob[i] > 0.5:
      y_class[i] = 1   # I classify as quasars all the objects with prob>0.5

c = confusion_matrix(Y_test, y_class)
print(c)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[847   7]
 [ 13 133]]


I have not obtained a better result.

I work on the hyperparameters. With keras_tuner I can find the best parameter to define the number of neurons in the hidden layer and I can identify the best value for the learning rate.

In [15]:
import keras_tuner as kt

def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(4,)))

  # Tune the number of units in the first Dense layer
  hp_units = hp.Int('units', min_value=5, max_value=50, step=5)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))  # output binario

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [16]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10)

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

c:\Users\marti\Anaconda\envs\astro\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
tuner.search(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid))

Trial 30 Complete [00h 00m 45s]
val_accuracy: 0.977142870426178

Best val_accuracy So Far: 0.9852041006088257
Total elapsed time: 00h 09m 09s


In [18]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 40 and the optimal learning rate for the optimizer is 0.01.



In [21]:
# Build the model with the optimal hyperparameters and train it on the data (I also increse the number of epochs)
model = tuner.hypermodel.build(best_hps)
clf = model.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))

y_prob = model.predict(X_test)   # Probability to be a quasar
y_class = np.zeros(len(y_prob))
for i in range(len(y_prob)):
   if y_prob[i] > 0.5:
      y_class[i] = 1   # I classify as quasars all the objects with prob>0.5

c = confusion_matrix(Y_test, y_class)
print(c)

c:\Users\marti\Anaconda\envs\astro\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9800 - loss: 0.0872 - val_accuracy: 0.9833 - val_loss: 0.0722
Epoch 2/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9829 - loss: 0.0800 - val_accuracy: 0.9841 - val_loss: 0.0658
Epoch 3/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9830 - loss: 0.0767 - val_accuracy: 0.9848 - val_loss: 0.0690
Epoch 4/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9836 - loss: 0.0730 - val_accuracy: 0.9821 - val_loss: 0.0673
Epoch 5/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9841 - loss: 0.0704 - val_accuracy: 0.9842 - val_loss: 0.0678
Epoch 6/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9837 - loss: 0.0680 - val_accuracy: 0.9840 - val_loss: 0.0717
Epoch 7/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9842 - loss: 0.0698 - val_accuracy: 0.9847 - val_loss: 0.0592
Epoch 8/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9839 - loss: 0.0659 - 

I still struggle to classify some data. Can another Neural Network do better? I try with MPLClassifier

# MPLclassifier

In [22]:
import sklearn.model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

I define the Neural Network

In [23]:
clf = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=(5), 
    activation='relu',
    solver='adam',
    alpha=0,
    learning_rate_init=0.001,
    max_iter=200)

# I divide my data in train and test sets, I don't perform cross-validation
# I'm just checking the performance of MPLClassifier
X_train = data_normed[:,:4]
Y_train = data_normed[:,4]
X_test = data_test[:,:4]
Y_test = data_test[:,4]

clf.fit(X_train, Y_train)

,hidden_layer_sizes,5
,activation,'relu'
,solver,'adam'
,alpha,0
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,None


I obtain predictions

In [24]:
y_prob=clf.predict_proba(X_test)[:,1]

y_class = np.zeros(len(y_prob))
for i in range(len(y_prob)):
   if y_prob[i] > 0.5:
      y_class[i] = 1   # I classify as quasars all the objects with prob>0.5
      
# confusion matrix
c = confusion_matrix(Y_test, y_class)
print(c)

[[847   7]
 [ 15 131]]


The result is similar to the one obtained with tensorflow. I try to tune the hyperparameters: I find the best hidden layer sizes and the best learning rate.

In [25]:
mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [5, 10, 15, 20],
    'learning_rate': ['constant','adaptive'],
}

from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, Y_train)

c:\Users\marti\Anaconda\envs\astro\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


,estimator,MLPClassifier(max_iter=100)
,param_grid,"{'hidden_layer_sizes': [5, 10, ...], 'learning_rate': ['constant', 'adaptive']}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,hidden_layer_sizes,20


In [26]:
y_prob=clf.predict_proba(X_test)[:,1]
y_class = np.zeros(len(y_prob))
for i in range(len(y_prob)):
   if y_prob[i] > 0.5:
      y_class[i] = 1   # I classify as quasars all the objects with prob>0.5
      
# confusion matrix
c = confusion_matrix(Y_test, y_class)
print(c)

[[849   5]
 [ 10 136]]


This is the best result, but there are still some misclassified stars. The accuracy is similar to the one obtained in Lecture19.